## Implementing ReAct Agent:

### <u>ReAct  Agent</u>:

It is a general agent, where the llm is the brain which calls the tools, observe the output along with the input and then resons whether to call the tools again or directly submit the output of the llm. It contains of main 3 components:

* **Act**: It calls the tool.
* **Observe** : The LLM observes the output of the tool.
* **Reasoning** : The LLM reasons whether to call the tool again or to end.

In [1]:
# from typing_extensions import TypedDict
# from typing import Annotated
# from langchain_core.messages import AnyMessage
# from langgraph.graph.message import add_messages

# class State(TypedDict):
#     messages:Annotated[list[AnyMessage],add_messages]


from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = 'llama3.2')

In [12]:
from datetime import datetime
datetime.now().time()

datetime.time(12, 57, 56, 880735)

In [13]:
from datetime import datetime

def date():
    """ Returns today's date, month and year
    args: none
    return: today's date, month and year which is a datetime object
    """
    return datetime.now().date()

def dt():
    """ Returns the current time
    args: none
    return: current time which is a datetime object
    """
    return datetime.now().time()

In [14]:
tools = [date,dt]

In [15]:
llm_with_tools = llm.bind_tools(tools)

In [16]:
def llm_tool(state:State):
    return {'messages':[llm_with_tools.invoke(state['messages'])]}

In [17]:
from langgraph.graph import START,END,StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [18]:
builder = StateGraph(State)

builder.add_node("llm_calling_tools",llm_tool)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"llm_calling_tools")
builder.add_conditional_edges('llm_calling_tools', tools_condition)
builder.add_edge("tools","llm_calling_tools")
builder.add_edge("llm_calling_tools",END)

builder_graph = builder.compile()

In [19]:
response = builder_graph.invoke({'messages':"I want to know the current time as well as today's date."})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know the current time as well as today's date.
================================== Ai Message ==================================
Tool Calls:
  dt (2965f1c7-5f36-408e-8f82-bb096226ef46)
 Call ID: 2965f1c7-5f36-408e-8f82-bb096226ef46
  Args:
================================= Tool Message =================================
Name: dt

12:58:20.730638
================================== Ai Message ==================================

The current time is 12:58:20 PM and the current date is Monday, May 6, 2024.
